In [1]:
import sys
if "../../" not in sys.path:
    sys.path.append("../../")

In [2]:
import pandas as pd
import numpy as np
from importlib import reload
from tqdm import tqdm_notebook as tqdm
import time

import pdaactconn as pc
from trialexplorer import AACTStudySet

import matplotlib.pyplot
%matplotlib inline

In [3]:
conn = pc.AACTConnection(source=pc.AACTConnection.REMOTE)
ss = AACTStudySet.AACTStudySet(conn=conn, 
                               tqdm_handler=tqdm)
ss.add_constraint("study_type = 'Interventional'")
ss.load_studies()

257905 studies loaded!


In [4]:
ss.add_dimensions(['baseline_measurements'])
ss.add_dimensions(['result_groups'])
ss.add_dimensions(['eligibilities'])
ss.add_dimensions(['outcomes'])
ss.add_dimensions(['outcome_measurements'])
ss.refresh_dim_data()

Successfuly added these 1 dimensions: ['baseline_measurements']
Failed to add these 0 dimensions: []
Successfuly added these 1 dimensions: ['result_groups']
Failed to add these 0 dimensions: []
Successfuly added these 1 dimensions: ['eligibilities']
Failed to add these 0 dimensions: []
Successfuly added these 1 dimensions: ['outcomes']
Failed to add these 0 dimensions: []
Successfuly added these 1 dimensions: ['outcome_measurements']
Failed to add these 0 dimensions: []


Syncing the temp table temp_cur_studies in 516 chunks x 500 records each

Creating index on the temp table
 - Loading dimension baseline_measurements
 -- Loading raw data
 -- Sorting index
 - Loading dimension result_groups
 -- Loading raw data
 -- Sorting index
 - Loading dimension eligibilities
 -- Loading raw data
 -- Sorting index
 - Loading dimension outcomes
 -- Loading raw data
 -- Sorting index
 - Loading dimension outcome_measurements
 -- Loading raw data
 -- Sorting index


In [5]:
num_all = ss.studies.shape[0]
num_all

257905

In [6]:
bm = ss.dimensions['baseline_measurements'].data
rg = ss.dimensions['result_groups'].data
el = ss.dimensions['eligibilities'].data
out = ss.dimensions['outcomes'].data
om = ss.dimensions['outcome_measurements'].data

## Where are all the places we can find gender information in bm?

In [7]:
bm.head()

id ctgov_group_code classification category  \
nct_id      result_group_id                                                     
NCT00000125 3090367          3368156               B3                    Male   
            3090367          3368159               B3                  Female   
            3090367          3368162               B3                           
            3090367          3368138               B3          Other            
            3090367          3368141               B3          White            

                                                  title description  \
nct_id      result_group_id                                           
NCT00000125 3090367                   Sex: Female, Male               
            3090367                   Sex: Female, Male               
            3090367                                 Age               
            3090367          Race/Ethnicity, Customized               
            3090367          Race/Ethnicity, Customized               

                                    units             param_type param_value  \
nct_id      result_group_id                                                    
NCT00000125 3090367          Participants  Count of Participants         931   
            3090367          Participants  Count of Participants         705   
            3090367                 years                   Mean        55.4   
            3090367          participants                 Number          14   
            3090367          participants                 Number        1137   

                             param_value_num     dispersion_type  \
nct_id      result_group_id                                        
NCT00000125 3090367                    931.0                       
            3090367                    705.0                       
            3090367                     55.4  Standard Deviation   
            3090367                     14.0                       
            3090367                   1137.0                       

                            dispersion_value  dispersion_value_num  \
nct_id      result_group_id                                          
NCT00000125 3090367                     None                   NaN   
            3090367                     None                   NaN   
            3090367                      9.6                   9.6   
            3090367                     None                   NaN   
            3090367                     None                   NaN   

                             dispersion_lower_limit  dispersion_upper_limit  \
nct_id      result_group_id                                                   
NCT00000125 3090367                             NaN                     NaN   
            3090367                             NaN                     NaN   
            3090367                             NaN                     NaN   
            3090367                             NaN                     NaN   
            3090367                             NaN                     NaN   

                            explanation_of_na  
nct_id      result_group_id                    
NCT00000125 3090367                            
            3090367                            
            3090367                            
            3090367                            
            3090367

In [8]:
bm.shape

(1087993, 16)

In [9]:
bm_all = bm.index.get_level_values(0).unique().shape[0]
bm_all

38296

<hr>

In [200]:
bm_all / num_all

0.14848878462999943

##### searching in the "classification" field

In [10]:
possible_classes = [x for x in bm['classification'].unique() if 'male' in x.lower()]

In [11]:
dfc = bm[bm['classification'].isin(possible_classes)]
dfc.shape, dfc.index.get_level_values(0).unique().shape[0]/bm_all

((5445, 16), 0.021621057029454775)

In [12]:
dfc['title'].unique()[:10]

array(['Gender', 'Sex/Gender, Customized',
       'Family History of Coronary Artery Disease', 'Visit type', 'Age',
       'History of Vomiting with Pregnancy',
       'Clayton Sexual Functioning Questionnaire (CSFQ)',
       'Gender at Beginning of Long term Follow-up Period',
       'Gender, by Disease Type', 'Donor gender'], dtype=object)

In [13]:
dfc.groupby('classification').size().sort_values(ascending=False)[:10]

classification
Female                                 1866
Male                                   1848
Female of Childbearing Potential        113
Females                                  57
Males                                    40
N/A (Subject is Male)                    30
Female of childbearing potential         30
Females of Child-bearing Potential       26
Sterilized Females                       26
Males (Excluded from this analysis)      26
dtype: int64

##### Use Female, Male and their plurals

In [14]:
used_classes = ['female', 'male', 'females', 'males']

In [15]:
used_dfc = dfc[dfc['classification'].apply(lambda x: x.lower()).isin(used_classes)]
res_studies_dfc = used_dfc.index.get_level_values(0).unique().shape[0]
res_studies_dfc

681

In [16]:
used_dfc['classification'].unique()

array(['Male', 'Female', 'male', 'Males', 'Females', 'female', 'FEMALE',
       'MALE', 'females', 'males'], dtype=object)

In [17]:
pct_classes = res_studies_dfc / bm_all
pct_classes

0.01778253603509505

##### conclusion: using the "classification" field for the gender infor gets us just under 2% of the 38k studies with baseline metrics, where else can we get gender information?

<hr>

##### searching in the "category" field

In [18]:
bm.groupby('category').size().sort_values(ascending=False)

category
                                          489196
Male                                      107991
Female                                    107973
Unknown or Not Reported                    41581
>=65 years                                 35307
                                           ...  
Age group at baseline: >12 to 18 years         1
Age group at baseline: >18 years               1
Age ≥ 60 years                                 1
Aggressive systemic mastocytosis (ASM)         1
Study provided device                          1
Length: 7697, dtype: int64

In [19]:
possible_class_cat = [x for x in bm['category'].unique() if 'male' in x.lower()]
possible_class_cat

['Male',
 'Female',
 'not applicable (male)',
 'Female-Female',
 'Male-Male',
 'Female-Male',
 'Male-Female',
 'Male patients',
 'High (>=102 cm for male and >=88 cm for female)',
 'Low (<102 cm for male and <88 cm for female)',
 'male',
 'female',
 'Male Donor, Male Recipient',
 'Male Donor, Female Recipient',
 'Female Donor, Male Recipient',
 'Female Donor, Female Recipient',
 'MALE',
 'FEMALE',
 'Transfemale',
 'Females',
 'Males',
 'Sex (Female Only)',
 'Male donor / Male Recipient',
 'Female donor / Female Recipient',
 'Female donor / Male Recipient',
 'Male donor / Female Recipient',
 'Female ≥4',
 'Female <4',
 'Male ≥8',
 'Male <8',
 'Female of Childbearing Potential',
 'Female of childbearing potential',
 'Not applicable (male participants)',
 'Female - shirt collar >=16 in/41 cm',
 'Female - shirt collar <16 in/41 cm',
 'Male - shirt collar >= 17 in/43 cm',
 'Male - shirt collar <17 in/43 cm',
 'Not Applicable (Male)',
 'Tubal and male factors',
 'Male factor',
 'Not applicab

In [20]:
# aside: what are those with both genders listed, such as 'Female-Male'?
cur_list = ['Female-Female',
'Male-Male',
'Female-Male',
'Male-Female',]

cur_df = bm[bm['category'].isin(cur_list)]
cur_df

# ah donor to recipients

id ctgov_group_code classification  \
nct_id      result_group_id                                            
NCT00448357 2950959          3218413               B1                  
            2950959          3218414               B1                  
            2950959          3218415               B1                  
            2950959          3218416               B1                  

                                  category                title description  \
nct_id      result_group_id                                                   
NCT00448357 2950959          Female-Female  Donor-recipient sex               
            2950959              Male-Male  Donor-recipient sex               
            2950959            Female-Male  Donor-recipient sex               
            2950959            Male-Female  Donor-recipient sex               

                                    units             param_type param_value  \
nct_id      result_group_id                                                    
NCT00448357 2950959          Participants  Count of Participants          14   
            2950959          Participants  Count of Participants          22   
            2950959          Participants  Count of Participants          12   
            2950959          Participants  Count of Participants           6   

                             param_value_num dispersion_type dispersion_value  \
nct_id      result_group_id                                                     
NCT00448357 2950959                     14.0                             None   
            2950959                     22.0                             None   
            2950959                     12.0                             None   
            2950959                      6.0                             None   

                             dispersion_value_num  dispersion_lower_limit  \
nct_id      result_group_id                                                 
NCT00448357 2950959                           NaN                     NaN   
            2950959                           NaN                     NaN   
            2950959                           NaN                     NaN   
            2950959                           NaN                     NaN   

                             dispersion_upper_limit explanation_of_na  
nct_id      result_group_id                                            
NCT00448357 2950959                             NaN                    
            2950959                             NaN                    
            2950959                             NaN                    
            2950959                             NaN

In [21]:
dfcat = bm[bm['category'].isin(possible_class_cat)]
dfcat.shape

(216230, 16)

In [22]:
dfcat.groupby('category').size().sort_values(ascending=False)[:6]

category
Male                                107991
Female                              107973
FEMALE                                  68
MALE                                    68
Female of childbearing potential        18
Not Applicable (Male)                    5
dtype: int64

Seems like if we just use "Male, Female" in any case, we will capture the vast majority of cases in this category

In [23]:
used_cats = ['male', 'female']
used_dfcat = bm[bm['category'].apply(lambda x: x.lower()).isin(used_cats)]

In [24]:
used_dfcat['category'].unique()

array(['Male', 'Female', 'male', 'female', 'MALE', 'FEMALE'], dtype=object)

In [25]:
pct_cats = np.unique(used_dfcat.index.get_level_values(0)).shape[0] / bm_all
pct_cats

0.9784311677459787

#### This seems a majority of our studies, is there overlap between identifying gender using classification vs using category?

Let's analyse their intersections

In [26]:
used_dfc['classification'].unique()

array(['Male', 'Female', 'male', 'Males', 'Females', 'female', 'FEMALE',
       'MALE', 'females', 'males'], dtype=object)

In [27]:
used_dfc.groupby('category').size().sort_values(ascending=False)

category
                           3849
Male                          7
Female                        4
Between 18 and 65 years       4
>=65 years                    4
<=18 years                    4
Transfemale                   3
dtype: int64

Most of the valid classifications have null as category, what about the others?

In [28]:
cur_to_investigate = used_dfc.groupby('category').size().sort_values(ascending=False)[3:].index
used_dfc[used_dfc['category'].isin(cur_to_investigate)]

id ctgov_group_code classification  \
nct_id      result_group_id                                            
NCT01781806 2384509          2641670               B3           Male   
            2384510          2641671               B2           Male   
            2384511          2641672               B1           Male   
NCT02844998 2115438          2314833               B4           Male   
            2115438          2314837               B4           Male   
            2115438          2314841               B4           Male   
            2115439          2314834               B3           Male   
            2115439          2314838               B3           Male   
            2115439          2314842               B3           Male   
            2115440          2314835               B2           Male   
            2115440          2314839               B2           Male   
            2115440          2314843               B2           Male   
            2115441          2314836               B1           Male   
            2115441          2314840               B1           Male   
            2115441          2314844               B1           Male   

                                            category                   title  \
nct_id      result_group_id                                                    
NCT01781806 2384509                      Transfemale  Sex/Gender, Customized   
            2384510                      Transfemale  Sex/Gender, Customized   
            2384511                      Transfemale  Sex/Gender, Customized   
NCT02844998 2115438                       >=65 years                     Age   
            2115438          Between 18 and 65 years                     Age   
            2115438                       <=18 years                     Age   
            2115439                       >=65 years                     Age   
            2115439          Between 18 and 65 years                     Age   
            2115439                       <=18 years                     Age   
            2115440                       >=65 years                     Age   
            2115440          Between 18 and 65 years                     Age   
            2115440                       <=18 years                     Age   
            2115441                       >=65 years                     Age   
            2115441          Between 18 and 65 years                     Age   
            2115441                       <=18 years                     Age   

                            description         units             param_type  \
nct_id      result_group_id                                                    
NCT01781806 2384509                      Participants  Count of Participants   
            2384510                      Participants  Count of Participants   
            2384511                      Participants  Count of Participants   
NCT02844998 2115438                      Participants  Count of Participants   
            2115438                      Participants  Count of Participants   
            2115438                      Participants  Count of Participants   
            2115439                      Participants  Count of Participants   
            2115439                      Participants  Count of Participants   
            2115439                      Participants  Count of Participants   
            2115440                      Participants  Count of Participants   
            2115440                      Participants  Count of Participants   
            2115440                      Participants  Count of Participants   
            2115441                      Participants  Count of Participants   
            2115441                      Participants  Count of Participants   
            2115441                      Participants  Count of Participants   

                            param_value  param_value_num dispersion_type  \
nct_id     

### looks like this is only 2 studies, one is on transgender and the other is male only so we can exclude from our balance analysis

In [29]:
used_dfcat['category'].unique()

array(['Male', 'Female', 'male', 'female', 'MALE', 'FEMALE'], dtype=object)

In [30]:
used_dfcat.groupby('classification').size().sort_values(ascending=False)

classification
                           214980
Part 2                         60
Sex                            58
Gender                         50
Part A                         34
                            ...  
Full analysis set (FAS)         2
GIST                            2
BiRD Phase                      2
Baseline Period Sex             2
Neuro-typical                   2
Length: 147, dtype: int64

In [31]:
used_dfcat.groupby('classification').size().sort_values(ascending=False) / used_dfcat.shape[0]

classification
                           0.994790
Part 2                     0.000278
Sex                        0.000268
Gender                     0.000231
Part A                     0.000157
                             ...   
Full analysis set (FAS)    0.000009
GIST                       0.000009
BiRD Phase                 0.000009
Baseline Period Sex        0.000009
Neuro-typical              0.000009
Length: 147, dtype: float64

##### This one looks like there is more crosses, but most of it still has '' - for simplicity let's just take the null studies

In [32]:
used_dfc_final = used_dfc[used_dfc['category'] == '']
used_dfc_final.shape

(3849, 16)

In [33]:
used_dfcat_final = used_dfcat[used_dfcat['classification'] == '']
used_dfcat_final.shape

(214980, 16)

### what about the overlap between the two sets?

In [34]:
i1 = used_dfc_final.index.get_level_values(0).unique()
i2 = used_dfcat_final.index.get_level_values(0).unique()

In [35]:
len(i1), len(i2), bm_all

(680, 37373, 38296)

In [36]:
len_overlap = len([x for x in i1 if x in i2])
len_overlap

42

In [37]:
overlap_nctids = [x for x in i1 if x in i2]
overlap_nctids[:5]

['NCT00095303', 'NCT00247611', 'NCT00284089', 'NCT00287469', 'NCT00340834']

Looks like 44 studies can be identified both ways

In [38]:
(len(i1) + len(i2) - len_overlap) / bm_all

0.9925579695007312

##### we can cover 99.3% of our studies with baseline measurements just using these 2 ways of identifying gender - this is good enough

In [39]:
used_dfc_final.index[2][0], used_dfcat_final.index[0][0]

('NCT00002525', 'NCT00000125')

In [40]:
used_dfc_final.loc['NCT00002525']

,id,ctgov_group_code,classification,category,title,description,units,param_type,param_value,param_value_num,dispersion_type,dispersion_value,dispersion_value_num,dispersion_lower_limit,dispersion_upper_limit,explanation_of_na
result_group_id,,,,,,,,,,,,,,,,
3077994,3350816,B3,Male,,Gender,"1 patient on ""No perioperative 5-FU arm"" had m...",participants,Number,195,195.0,,None,NaN,NaN,NaN,
3077994,3350819,B3,Female,,Gender,"1 patient on ""No perioperative 5-FU arm"" had m...",participants,Number,118,118.0,,None,NaN,NaN,NaN,
3077995,3350817,B2,Male,,Gender,"1 patient on ""No perioperative 5-FU arm"" had m...",participants,Number,96,96.0,,None,NaN,NaN,NaN,
3077995,3350820,B2,Female,,Gender,"1 patient on ""No perioperative 5-FU arm"" had m...",participants,Number,61,61.0,,None,NaN,NaN,NaN,
3077996,3350818,B1,Male,,Gender,"1 patient on ""No perioperative 5-FU arm"" had m...",participants,Number,99,99.0,,None,NaN,NaN,NaN,
3077996,3350821,B1,Female,,Gender,"1 patient on ""No perioperative 5-FU arm"" had m...",participants,Number,57,57.0,,None,NaN,NaN,NaN,


In [41]:
rg.loc['NCT00002525']

,id,ctgov_group_code,result_type,title,description
nct_id,,,,,
NCT00002525,3077994,B3,Baseline,Total,Total of all reporting groups
NCT00002525,3077995,B2,Baseline,No Perioperative 5-FU,Patients receive no perioperative fluorouracil...
NCT00002525,3077996,B1,Baseline,Perioperative 5-FU,"Within 24 hours of the colon resection, patien..."
NCT00002525,3077997,P2,Participant Flow,No Perioperative 5-FU,Patients receive no perioperative fluorouracil...
NCT00002525,3077998,P1,Participant Flow,Perioperative 5-FU,"Within 24 hours of the colon resection, patien..."
NCT00002525,3077999,O2,Outcome,No Perioperative 5-FU,Patients receive no perioperative fluorouracil...
NCT00002525,3078000,O1,Outcome,Perioperative 5-FU,"Within 24 hours of the colon resection, patien..."
NCT00002525,3078001,O2,Outcome,No Perioperative 5-FU,Patients receive no perioperative fluorouracil...
NCT00002525,3078002,O1,Outcome,Perioperative 5-FU,"Within 24 hours of the colon resection, patien..."


### let's build the gender information df using these 2 methods

In [42]:
used_df_both = pd.concat([used_dfc_final, used_dfcat_final])
used_df_both.shape

(218829, 16)

In [43]:
used_df_both['classification'].unique()

array(['Male', 'Female', 'male', 'Males', 'Females', 'female', 'FEMALE',
       'MALE', 'females', 'males', ''], dtype=object)

In [44]:
used_classes

['female', 'male', 'females', 'males']

In [45]:
used_df_both['category'].unique()

array(['', 'Male', 'Female', 'male', 'female', 'MALE', 'FEMALE'],
      dtype=object)

In [46]:
used_cats

['male', 'female']

In [47]:
def is_male(row):
    cur_class = row['classification']
    cur_cat = row['category']
    if 'female' in cur_class.lower() or 'female' in cur_cat.lower():
        return False
    return True

In [48]:
used_df_both['is_male'] = used_df_both.apply(is_male, axis=1)

In [49]:
used_df_both.head()

id ctgov_group_code classification category  \
nct_id      result_group_id                                                     
NCT00001213 3090179          3367810               B1           Male            
            3090179          3367811               B1         Female            
NCT00002525 3077994          3350816               B3           Male            
            3077994          3350819               B3         Female            
            3077995          3350817               B2           Male            

                              title  \
nct_id      result_group_id           
NCT00001213 3090179          Gender   
            3090179          Gender   
NCT00002525 3077994          Gender   
            3077994          Gender   
            3077995          Gender   

                                                                   description  \
nct_id      result_group_id                                                      
NCT00001213 3090179          Gender was captured for 326 out of 328 partici...   
            3090179          Gender was captured for 326 out of 328 partici...   
NCT00002525 3077994          1 patient on "No perioperative 5-FU arm" had m...   
            3077994          1 patient on "No perioperative 5-FU arm" had m...   
            3077995          1 patient on "No perioperative 5-FU arm" had m...   

                                    units param_type param_value  \
nct_id      result_group_id                                        
NCT00001213 3090179          participants     Number         173   
            3090179          participants     Number         153   
NCT00002525 3077994          participants     Number         195   
            3077994          participants     Number         118   
            3077995          participants     Number          96   

                             param_value_num dispersion_type dispersion_value  \
nct_id      result_group_id                                                     
NCT00001213 3090179                    173.0                             None   
            3090179                    153.0                             None   
NCT00002525 3077994                    195.0                             None   
            3077994                    118.0                             None   
            3077995                     96.0                             None   

                             dispersion_value_num  dispersion_lower_limit  \
nct_id      result_group_id                                                 
NCT00001213 3090179                           NaN                     NaN   
            3090179                           NaN                     NaN   
NCT00002525 3077994                           NaN                     NaN   
            3077994                           NaN                     NaN   
            3077995                           NaN                     NaN   

                             dispersion_upper_limit explanation_of_na  is_male  
nct_id      result_group_id                                                     
NCT00001213 3090179                             NaN                       True  
            3090179                             NaN                      False  
NCT00002525 3077994                             NaN                       True  
            3077994                             NaN                      False  
            3077995                             NaN                       True

### Q1: what is the distribution of studies by gender eligibility?

In [50]:
# is the index on eligibility unique?
el.index.shape, el.index.unique().shape

# Yes

((257905,), (257905,))

In [51]:
el.groupby('gender').size()/el.shape[0]

gender
          0.000050
All       0.850689
Female    0.097679
Male      0.051581
dtype: float64

### Q2: if the eligibility is only by gender, do the studies accurately recruit that gender?

In [52]:
el_female = el[el['gender'] == 'Female']
el_female.shape

(25192, 10)

In [53]:
df_female = el_female[[]].join(used_df_both)
df_female.head()

id ctgov_group_code classification category  \
nct_id      result_group_id                                                     
NCT00004888 3085562          3362332               B3         Female            
            3085563          3362333               B2         Female            
            3085564          3362334               B1         Female            
NCT00062751 3082311          3357259               B6         Female            
            3082312          3357260               B5         Female            

                                              title  \
nct_id      result_group_id                           
NCT00004888 3085562          Sex/Gender, Customized   
            3085563          Sex/Gender, Customized   
            3085564          Sex/Gender, Customized   
NCT00062751 3082311          Sex/Gender, Customized   
            3082312          Sex/Gender, Customized   

                                              description         units  \
nct_id      result_group_id                                               
NCT00004888 3085562          All participants are female.  participants   
            3085563          All participants are female.  participants   
            3085564          All participants are female.  participants   
NCT00062751 3082311                                        participants   
            3082312                                        participants   

                            param_type param_value  param_value_num  \
nct_id      result_group_id                                           
NCT00004888 3085562             Number          84             84.0   
            3085563             Number          46             46.0   
            3085564             Number          38             38.0   
NCT00062751 3082311             Number         108            108.0   
            3082312             Number           6              6.0   

                            dispersion_type dispersion_value  \
nct_id      result_group_id                                    
NCT00004888 3085562                                     None   
            3085563                                     None   
            3085564                                     None   
NCT00062751 3082311                                     None   
            3082312                                     None   

                             dispersion_value_num  dispersion_lower_limit  \
nct_id      result_group_id                                                 
NCT00004888 3085562                           NaN                     NaN   
            3085563                           NaN                     NaN   
            3085564                           NaN                     NaN   
NCT00062751 3082311                           NaN                     NaN   
            3082312                           NaN                     NaN   

                             dispersion_upper_limit explanation_of_na  is_male  
nct_id      result_group_id                                                     
NCT00004888 3085562                             NaN                      False  
            3085563                             NaN                      False  
            3085564                             NaN                      False  
NCT00062751 3082311                             NaN                      False  
            3082312                             NaN                      False

In [54]:
df_male_in_f_study = df_female[df_female['is_male']]

In [55]:
df_non_zero = df_male_in_f_study[(df_male_in_f_study['param_value'] != '0') & 
                                 (df_male_in_f_study['param_value'] != '0.0') &
                                 (df_male_in_f_study['param_value'] != 'NA')]

In [56]:
df_non_zero.index.get_level_values(0).unique()

Index(['NCT03628885', 'NCT00080301', 'NCT00275301', 'NCT00291577',
       'NCT00394524', 'NCT00412841', 'NCT00421889', 'NCT00719862',
       'NCT00743119', 'NCT00810108', 'NCT00846027', 'NCT00874094',
       'NCT00878995', 'NCT00920621', 'NCT01032135', 'NCT01121250',
       'NCT01147744', 'NCT01156792', 'NCT01181323', 'NCT01193920',
       'NCT01211340', 'NCT01251744', 'NCT01269346', 'NCT01412801',
       'NCT01443130', 'NCT01446289', 'NCT01615367', 'NCT01722266',
       'NCT01737710', 'NCT01801475', 'NCT01919216', 'NCT02078713',
       'NCT02175628', 'NCT02244424', 'NCT02263118', 'NCT02415166',
       'NCT02487771', 'NCT02721147', 'NCT03098615'],
      dtype='object', name='nct_id')

- Looks like some of the results included the genders of childern of those studied: NCT03628885
- sometimes despite a female study such as breast cancer, we get a odd male participant: NCT00080301
- sometimes the implementation is different from design, resulting in many males: NCT00275301

Take away: we should limit it to studies that intended to study both genders at the start, which represents 85% of our study pop

In [57]:
df_all_gender_idx = el[el['gender'] == 'All'][[]]

In [201]:
df_all_gender_idx

""
nct_id
NCT00000102
NCT00000106
NCT00000110
NCT00000111
NCT00000113
...
NCT04230889
NCT04230902
NCT04230915


### rebuilding the joint gender dataframe

In [58]:
used_dfc_final2 = used_dfc_final.join(df_all_gender_idx, how='inner')
used_dfc_final.shape[0], used_dfc_final2.shape[0]

(3849, 3092)

In [59]:
used_dfcat_final2 = used_dfcat_final.join(df_all_gender_idx, how='inner')
used_dfcat_final.shape[0], used_dfcat_final2.shape[0]

(214980, 188856)

In [60]:
used_df_both = pd.concat([used_dfc_final2, used_dfcat_final2])
used_df_both.shape

(191948, 16)

In [61]:
used_df_both['is_male'] = used_df_both.apply(is_male, axis=1)

### removing odd param_types

In [62]:
used_df_both.groupby('param_type').size()

param_type
                             31
Count of Participants    189080
Count of Units              123
Mean                         74
Median                        6
Number                     2634
dtype: int64

In [63]:
used_df_both = used_df_both[used_df_both['param_type'].isin(['Count of Participants','Number'])]

In [64]:
num_remain = used_df_both.index.get_level_values(0).unique().shape[0]
num_remain, num_remain / bm_all

(33164, 0.8659912262377272)

##### we still have 33k and 87% of our studies that we started with in baseline measurements

### Joining to result groups to get the group labels

In [65]:
dfm = used_df_both.reset_index().merge(rg.reset_index()[['nct_id', 'ctgov_group_code', 'title']], 
                                       on=['nct_id', 'ctgov_group_code'], 
                                       how='inner', 
                                       suffixes=('', '_rg'))

In [66]:
dfm.groupby('title_rg').size().sort_values(ascending=False)[:50].index

Index(['Total', 'Placebo', 'Control', 'Control Group', 'All Participants',
       'All Study Participants', 'Usual Care', 'Entire Study Population',
       'Overall Study', 'Treatment', 'Intervention', 'Placebo Group',
       'Group 1', 'Standard of Care', 'Overall', 'Arm 1', 'Vehicle', 'Group 2',
       'Sugar Pill', 'All Subjects', 'Arm 2', 'Cohort 1', 'Cohort 2', 'Arm I',
       'Total Title', 'Group B', 'All Patients', 'Standard Care', 'Group A',
       'Control Arm', 'Sitagliptin', 'Pregabalin', 'Varenicline',
       'Intervention Group', 'Experimental', 'Normal Saline', 'Duloxetine',
       'Cohort 3', 'Etanercept', 'Saline', 'Metformin', 'Treatment Arm',
       'Sham', 'Arm A', 'Arm B', 'Erlotinib', 'Active', 'Treatment Group',
       'Rituximab', 'Pioglitazone'],
      dtype='object', name='title_rg')

In [67]:
total_strings = ['Total', 
                 'All Participants', 
                 'All Study Participants', 
                 'Entire Study Population',
                 'Overall Study',
                 'Overall', 
                 'Total Title']

- exclude the 'percent' unit types
- just exclude the total group up front
- remove studies where the number of people < x
- then remove studies where there is only 1 non-zero group

In [68]:
dfm.shape

(191714, 20)

### Excluding total groups

In [69]:
dfm_nontot = dfm[~dfm['title_rg'].isin(total_strings)]
dfm_nontot.shape

(143802, 20)

### excluding percent unit types

In [128]:
dfm_nontot

,nct_id,result_group_id,id,ctgov_group_code,classification,category,title,description,units,param_type,param_value,param_value_num,dispersion_type,dispersion_value,dispersion_value_num,dispersion_lower_limit,dispersion_upper_limit,explanation_of_na,is_male,title_rg
0,NCT00001213,3090179,3367810,B1,Male,,Gender,Gender was captured for 326 out of 328 partici...,participants,Number,173,173.0,,None,NaN,NaN,NaN,,True,Cysteamine Topical Solution
1,NCT00001213,3090179,3367811,B1,Female,,Gender,Gender was captured for 326 out of 328 partici...,participants,Number,153,153.0,,None,NaN,NaN,NaN,,False,Cysteamine Topical Solution
4,NCT00002525,3077995,3350817,B2,Male,,Gender,"1 patient on ""No perioperative 5-FU arm"" had m...",participants,Number,96,96.0,,None,NaN,NaN,NaN,,True,No Perioperative 5-FU
5,NCT00002525,3077995,3350820,B2,Female,,Gender,"1 patient on ""No perioperative 5-FU arm"" had m...",participants,Number,61,61.0,,None,NaN,NaN,NaN,,False,No Perioperative 5-FU
6,NCT00002525,3077996,3350818,B1,Male,,Gender,"1 patient on ""No perioperative 5-FU arm"" had m...",participants,Number,99,99.0,,None,NaN,NaN,NaN,,True,Perioperative 5-FU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191709,NCT04144088,2058931,2243868,B3,,Female,"Sex: Female, Male",,Participants,Count of Participants,1,1.0,,None,NaN,NaN,NaN,,False,Placebo
191710,NCT04144088,2058932,2243865,B2,,Male,"Sex: Female, Male",,Participants,Count of Participants,18,18.0,,None,NaN,NaN,NaN,,True,Yin-Chen Wu Ling San
191711,NCT04144088,2058932,2243869,B2,,Female,"Sex: Female, Male",,Participants,Count of Participants,2,2.0,,None,NaN,NaN,NaN,,False,Yin-Chen Wu Ling San
191712,NCT04144088,2058933,2243866,B1,,Male,"Sex: Female, Male",,Participants,Count of Participants,18,18.0,,None,NaN,NaN,NaN,,True,Wu Ling San


In [70]:
dfm2 = dfm_nontot[dfm_nontot['units'].apply(lambda x: False if 'percent' in x.lower() else True)]
dfm2.shape

(143790, 20)

### Total participants:

In [71]:
dftotals = dfm2.groupby('nct_id')[['param_value_num']].sum()
dftotals.head()

,param_value_num
nct_id,
NCT00000125,1636.0
NCT00000134,279.0
NCT00000135,209.0
NCT00000136,234.0
NCT00000142,64.0


In [72]:
for i in range(10, 25):
    sub_df = dftotals[dftotals['param_value_num'] <= i]
    print(i, sub_df.shape[0], sub_df.shape[0]/dftotals.shape[0])

10 2954 0.09185037778676036
11 3227 0.10033891980970741
12 3637 0.11308727962438979
13 3885 0.12079848263424645
14 4169 0.12962905382295326
15 4486 0.1394857125089394
16 4824 0.14999533596592146
17 5049 0.15699138708373495
18 5369 0.16694132645129195
19 5611 0.1744659680980069
20 6215 0.1932464786542707
21 6496 0.20198376916140667
22 6758 0.21013028201859396
23 6998 0.21759273654426167
24 7415 0.23055875128260936


##### Decision: exclude any studies with less than 20 people

In [73]:
df_totals_used = dftotals[dftotals['param_value_num'] >= 20]
df_totals_used.shape

(26550, 1)

In [74]:
# excluding studies with < 30 total participants
dfm3 = dfm2[dfm2['nct_id'].isin(df_totals_used.index)]
dfm3.shape

(127200, 20)

## what are the studies that do not have atleast 2 non-zero groups?

In [75]:
dfm3.head()

,nct_id,result_group_id,id,ctgov_group_code,classification,category,title,description,units,param_type,param_value,param_value_num,dispersion_type,dispersion_value,dispersion_value_num,dispersion_lower_limit,dispersion_upper_limit,explanation_of_na,is_male,title_rg
0,NCT00001213,3090179,3367810,B1,Male,,Gender,Gender was captured for 326 out of 328 partici...,participants,Number,173,173.0,,None,NaN,NaN,NaN,,True,Cysteamine Topical Solution
1,NCT00001213,3090179,3367811,B1,Female,,Gender,Gender was captured for 326 out of 328 partici...,participants,Number,153,153.0,,None,NaN,NaN,NaN,,False,Cysteamine Topical Solution
4,NCT00002525,3077995,3350817,B2,Male,,Gender,"1 patient on ""No perioperative 5-FU arm"" had m...",participants,Number,96,96.0,,None,NaN,NaN,NaN,,True,No Perioperative 5-FU
5,NCT00002525,3077995,3350820,B2,Female,,Gender,"1 patient on ""No perioperative 5-FU arm"" had m...",participants,Number,61,61.0,,None,NaN,NaN,NaN,,False,No Perioperative 5-FU
6,NCT00002525,3077996,3350818,B1,Male,,Gender,"1 patient on ""No perioperative 5-FU arm"" had m...",participants,Number,99,99.0,,None,NaN,NaN,NaN,,True,Perioperative 5-FU


In [76]:
df_nonzero = dfm3[dfm3['param_value_num'] > 0]
df_nonzero.shape

(123859, 20)

In [77]:
df_nonzero_groups = df_nonzero.groupby(['nct_id'])['ctgov_group_code'].nunique()
df_nonzero_groups.shape

(26550,)

In [78]:
df_nonzero_groups[df_nonzero_groups <= 1]

nct_id
NCT00001213    1
NCT00001305    1
NCT00001566    1
NCT00001575    1
NCT00001941    1
              ..
NCT03783780    1
NCT03870737    1
NCT03905642    1
NCT03927950    1
NCT04036838    1
Name: ctgov_group_code, Length: 5953, dtype: int64

In [79]:
df_nonzero_used = df_nonzero_groups[df_nonzero_groups > 1].index
len(df_nonzero_used)

20597

In [80]:
dfm4 = dfm3[dfm3['nct_id'].isin(df_nonzero_used)]
dfm4.shape

(115232, 20)

In [81]:
len(dfm4['nct_id'].unique()) / bm_all, len(dfm4['nct_id'].unique()) 

(0.5378368498015459, 20597)

## we have 54% or 21k of the studies remaining that have:
- atleast 2 non-zero non-total groups
- not measured in percent
- have atleast a total of 20 participants

In [82]:
dfm4.to_pickle('balance_curated.p')

## Notes to detect imbalance issues:

if the treatment effect is large, then the expectation of absolute value of effect should be around the true effect regardless of imbalance or no imbalance. On the otherhand if the treatment effect is small, then the expectation of absolute value of the treatmentment effect is larger for imbalanced datasets compared to balanced datasets

The what we need to do:
- create universe of studies
- measure imbalance on each study
- measure treatment effect
- compare the average treatment effect on a grid of studies:
    - high/low treatment effect 
    - high/low imbalance

In [134]:
dfm4.head()

,nct_id,result_group_id,id,ctgov_group_code,classification,category,title,description,units,param_type,param_value,param_value_num,dispersion_type,dispersion_value,dispersion_value_num,dispersion_lower_limit,dispersion_upper_limit,explanation_of_na,is_male,title_rg
4,NCT00002525,3077995,3350817,B2,Male,,Gender,"1 patient on ""No perioperative 5-FU arm"" had m...",participants,Number,96,96.0,,None,NaN,NaN,NaN,,True,No Perioperative 5-FU
5,NCT00002525,3077995,3350820,B2,Female,,Gender,"1 patient on ""No perioperative 5-FU arm"" had m...",participants,Number,61,61.0,,None,NaN,NaN,NaN,,False,No Perioperative 5-FU
6,NCT00002525,3077996,3350818,B1,Male,,Gender,"1 patient on ""No perioperative 5-FU arm"" had m...",participants,Number,99,99.0,,None,NaN,NaN,NaN,,True,Perioperative 5-FU
7,NCT00002525,3077996,3350821,B1,Female,,Gender,"1 patient on ""No perioperative 5-FU arm"" had m...",participants,Number,57,57.0,,None,NaN,NaN,NaN,,False,Perioperative 5-FU
10,NCT00003641,3085820,3362852,B2,Male,,"Sex/Gender, Customized",1 patient on arm I and 1 patient on arm II had...,participants,Number,336,336.0,,None,NaN,NaN,NaN,,True,Interferon Alfa-2b


In [136]:
studies_w_gender_info = dfm4['nct_id'].unique()

# Constructing the Outcome dataset

##### Note: studies can have many primary outcome measurements: NCT01094184

- measure a monotonicly increasing balance function
- measure a monotonically increasing absolute effect size function
- The studies where the treatment effect is large should not be affect by impacted by the imbalance
- BUT the studies where the treatment effect is small, we should see a postive correlation to imbalance

## For our baseline, let's just take the most restrictive study set, although it may be small
- 2 outcome measure ctgov_group_codes
- 1 primary outcome measure

In [161]:
len(studies_w_gender_info)

20597

### getting studies with only 2 group codes

In [156]:
num_grp_per_study = omj.groupby('om_nct_id')['om_ctgov_group_code'].nunique()

In [180]:
studies_w_2_grps = num_grp_per_study[num_grp_per_study == 2].index

In [181]:
len(studies_w_2_grps)

18138

In [169]:
outj_primary = outj[outj['out_outcome_type'] == 'Primary']

In [172]:
num_outcomes_per_study = outj_primary.groupby('out_nct_id')['out_id'].size()

In [178]:
studies_w_1_outcome = num_outcomes_per_study[num_outcomes_per_study == 1].index

In [179]:
len(num_outcomes_per_study)

38459

### Joining Outcome and Outcome Measurements

In [107]:
omj = om.reset_index().add_prefix('om_')
outj = out.reset_index().add_prefix('out_')

In [109]:
omj.columns

Index(['om_nct_id', 'om_result_group_id', 'om_outcome_id', 'om_id',
       'om_ctgov_group_code', 'om_classification', 'om_category', 'om_title',
       'om_description', 'om_units', 'om_param_type', 'om_param_value',
       'om_param_value_num', 'om_dispersion_type', 'om_dispersion_value',
       'om_dispersion_value_num', 'om_dispersion_lower_limit',
       'om_dispersion_upper_limit', 'om_explanation_of_na'],
      dtype='object')

In [108]:
outj.columns

Index(['out_nct_id', 'out_id', 'out_outcome_type', 'out_title',
       'out_description', 'out_time_frame', 'out_population',
       'out_anticipated_posting_date', 'out_anticipated_posting_month_year',
       'out_units', 'out_units_analyzed', 'out_dispersion_type',
       'out_param_type'],
      dtype='object')

In [111]:
dfj = omj.merge(outj, 
                how='inner',
                left_on=['om_nct_id', 'om_outcome_id'],
                right_on=['out_nct_id', 'out_id'],)

In [182]:
dfj_used = dfj[(dfj['om_nct_id'].isin(studies_w_gender_info)) &
               (dfj['om_nct_id'].isin(studies_w_2_grps)) & 
               (dfj['om_nct_id'].isin(studies_w_1_outcome))]

In [188]:
num_left = len(dfj_used['om_nct_id'].unique())
num_left, num_left / bm_all

(8995, 0.2348809275120117)

In [189]:
df_prim_mean = dfj_used[(dfj_used['out_param_type'] == 'Mean') & (dfj_used['out_outcome_type'] == 'Primary')]
df_prim_mean.head()

,om_nct_id,om_result_group_id,om_outcome_id,om_id,om_ctgov_group_code,om_classification,om_category,om_title,om_description,om_units,...,out_title,out_description,out_time_frame,out_population,out_anticipated_posting_date,out_anticipated_posting_month_year,out_units,out_units_analyzed,out_dispersion_type,out_param_type
22,NCT00000371,3090202,930699,7024090,O2,,,Scale for the Assessment of Negative Symptoms ...,The slope of SANS total score from baseline to...,units on a scale/weeks,...,Scale for the Assessment of Negative Symptoms ...,The slope of SANS total score from baseline to...,"Baseline, Week 4, Week 8",,None,,units on a scale/weeks,,Standard Error,Mean
23,NCT00000371,3090203,930699,7024091,O1,,,Scale for the Assessment of Negative Symptoms ...,The slope of SANS total score from baseline to...,units on a scale/weeks,...,Scale for the Assessment of Negative Symptoms ...,The slope of SANS total score from baseline to...,"Baseline, Week 4, Week 8",,None,,units on a scale/weeks,,Standard Error,Mean
24,NCT00000378,3090193,930698,7024086,O2,12 WEEK DATA,,HAMILTON Rating Scale for DEPRESSION Range,"Hamilton scale range 0-40, values below 7 are ...",units on a scale,...,HAMILTON Rating Scale for DEPRESSION Range,"Hamilton scale range 0-40, values below 7 are ...",BASELINE COMPARED TO 12 WEEK MEASUREMENT,intent to treat analysis,None,,units on a scale,,Standard Deviation,Mean
25,NCT00000378,3090193,930698,7024088,O2,BASELINE,,HAMILTON Rating Scale for DEPRESSION Range,"Hamilton scale range 0-40, values below 7 are ...",units on a scale,...,HAMILTON Rating Scale for DEPRESSION Range,"Hamilton scale range 0-40, values below 7 are ...",BASELINE COMPARED TO 12 WEEK MEASUREMENT,intent to treat analysis,None,,units on a scale,,Standard Deviation,Mean
26,NCT00000378,3090194,930698,7024087,O1,12 WEEK DATA,,HAMILTON Rating Scale for DEPRESSION Range,"Hamilton scale range 0-40, values below 7 are ...",units on a scale,...,HAMILTON Rating Scale for DEPRESSION Range,"Hamilton scale range 0-40, values below 7 are ...",BASELINE COMPARED TO 12 WEEK MEASUREMENT,intent to treat analysis,None,,units on a scale,,Standard Deviation,Mean


In [190]:
df_prim_mean.columns

Index(['om_nct_id', 'om_result_group_id', 'om_outcome_id', 'om_id',
       'om_ctgov_group_code', 'om_classification', 'om_category', 'om_title',
       'om_description', 'om_units', 'om_param_type', 'om_param_value',
       'om_param_value_num', 'om_dispersion_type', 'om_dispersion_value',
       'om_dispersion_value_num', 'om_dispersion_lower_limit',
       'om_dispersion_upper_limit', 'om_explanation_of_na', 'out_nct_id',
       'out_id', 'out_outcome_type', 'out_title', 'out_description',
       'out_time_frame', 'out_population', 'out_anticipated_posting_date',
       'out_anticipated_posting_month_year', 'out_units', 'out_units_analyzed',
       'out_dispersion_type', 'out_param_type'],
      dtype='object')

In [191]:
df_prim_mean.shape

(11448, 32)

In [203]:
num_left = len(df_prim_mean['om_nct_id'].unique())

num_left / bm_all

0.08739816168790474

In [204]:
num_left

3347

In [199]:
df_prim_mean.to_pickle('curated_outcomes.p')